In [10]:
import sys
sys.path.append("..")
from pathlib import Path
from utils import save_jump_video
import pandas as pd
from scipy.stats import zscore, percentileofscore
from scipy.ndimage import gaussian_filter1d

In [22]:
# home_dir = Path(".").absolute().parent
home_dir = Path(r"C:\Users\rory\repos\ah_vids\output\PL_BLA")

jump_dir = home_dir / "jump_vids"
jump_dir.mkdir(exist_ok=True)

data_dir = home_dir / "reports"

vid_dir = Path(r"C:\Users\rory\repos\ah_vids\raw_videos\PL-ChR_OFL") / home_dir.name


In [23]:
vid_paths = data_dir.glob("*_dem.csv")

frames = []
for p in vid_paths:
    frames.append(pd.read_csv(p))
    
df = pd.concat(frames)

In [24]:
vids = df.video_name.unique()

In [25]:
dfz = (
    df
    .dropna()
    .assign(
        distance_z= lambda x: x.groupby("video_name")["distance"].transform(lambda x: zscore(x)),
        was_jump= lambda x: x.distance_z.apply(lambda y: y > 1.7).astype(int),
           )
    
)

In [6]:
def save_vid(df, vid_name, colname="was_jump"):
    source_file = str(vid_dir / vid_name)
    outfile_path = str(jump_dir / f"jump_{Path(vid_name).stem}.avi")
    freezes = df.loc[lambda x: x.video_name == vid_name][colname].values
    start_frame = df.loc[lambda x: x.video_name == vid_name]["frame"].min()
    
    save_jump_video(source_file, outfile_path=outfile_path, freezes=freezes, start_frame=start_frame)

In [14]:
vids = df.video_name.unique()

In [15]:
# save_vid(dfz, vids[3])

In [16]:
vids = df.video_name.unique()
for i, vid in enumerate(vids):
    print(str(((i + 1) / len(vids) * 100)) + "%")
    save_vid(dfz, vid, "was_jump")

33.33333333333333%
66.66666666666666%
100.0%


In [104]:
# save_vid(dfz, vid, "was_jump")

In [26]:
(
    dfz
    .to_csv(home_dir / "dem_reports_with_jumps.csv", index=False)
)